In [3]:
import selenium#webdriver is our tool to interact with the webpage
from selenium import webdriver
import requests #needed to load the page for BS4
from bs4 import BeautifulSoup
import pandas as pd #Using panda to create our dataframe

In [4]:
# !pip install webdriver-manager

In [5]:
from selenium.webdriver.firefox.service import Service

from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [39]:
driver_path = ChromeDriverManager().install()            

In [83]:
def get_link_top_100_film():
    options = Options()
    options.add_argument("start-maximized")

    driver = webdriver.Chrome(executable_path=driver_path)
    driver.get("https://www.imdb.com/chart/moviemeter/?sort=ir,desc&mode=simple&page=1")
    elements = driver.find_elements(By.CLASS_NAME,"titleColumn")
    links = []
    for element in elements:
        lnk = element.find_element(By.TAG_NAME,"a")
        links.append(lnk.get_attribute("href"))
    return links

In [82]:
top_100_links_film = get_link_top_100_film()
top_100_links_film

<ipython-input-81-2f442d2f35b7>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


['https://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=DS5Y5X6YXXBQ6V96QJ5R&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_75',
 'https://www.imdb.com/title/tt13833688/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=DS5Y5X6YXXBQ6V96QJ5R&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_1',
 'https://www.imdb.com/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=DS5Y5X6YXXBQ6V96QJ5R&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_54',
 'https://www.imdb.com/title/tt0167260/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=DS5Y5X6YXXBQ6V96QJ5R&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_53',
 'https://www.imdb.com/title/tt0120737/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=DS5Y5X6YXXBQ6V96QJ5R&pf_rd_s=center-1&pf_rd_t=15506&pf_r

In [98]:
def get_to_reviews_page(link):
    
    options = Options()
    options.add_argument("start-maximized")

    driver = webdriver.Chrome(executable_path=driver_path)
    driver.get(link)
    try:
        div = driver.find_element(By.CLASS_NAME,"ghlYSH")
        lnk = div.find_element(By.TAG_NAME,"a")
        print(lnk.get_attribute("href"))
        return lnk.get_attribute("href")
    except:
        return ''


In [99]:
reviews_pages = []
for film in top_100_links_film:
    reviews_pages.append(get_to_reviews_page(film))

<ipython-input-98-a5036773e837>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


https://www.imdb.com/title/tt0111161/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt13833688/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt0068646/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt0167260/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt0120737/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt0167261/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt0816692/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt1745960/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt10872600/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt11813216/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt6710474/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt0073195/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt11564570/externalreviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt1160419/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt1877830/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt8946378/reviews?ref_=tt_ov_rt
https://www.imdb.com/title/tt17076046/extern

In [101]:
reviews_pages_df = pd.DataFrame(reviews_pages)

In [11]:
reviews_pages_df.to_csv("link_reviews.csv",index=False, sep='|') # Just to save the result
#reviews_pages_df = pd.read_csv("link_reviews.csv", sep="|")

In [1]:
def get_reviews(rate, link):
    options = Options()
    options.add_argument("start-maximized")

    rate = str(rate)

    driver = webdriver.Chrome(executable_path=driver_path)
    
    try:
        driver.get(link)
        review = driver.find_element(By.CLASS_NAME,"lister-item")

        review_stars = Select(driver.find_element(By.NAME,"ratingFilter"))
                      
        review_stars.select_by_value(rate)

        load_more_reviews = driver.find_element(By.ID,"load-more-trigger")
    
        while(load_more_reviews.is_displayed()):
            load_more_reviews.click()
            try:
                WebDriverWait(driver, timeout=3).until(EC.element_to_be_clickable((By.ID,"load-more-trigger")))
            except:
                continue
        
    
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        mydivs = soup.find_all("div", {"class": "text show-more__control"})
        return mydivs
    except:
        return []
   

In [2]:
def transform_to_dataframe(rate, link):
    
    rev = get_reviews(rate, link)
    reviews = list(map(lambda x:x.text, rev))
    
    reviews_df = pd.DataFrame(reviews, columns=["review"])
    reviews_df["rating"] = rate
    reviews_df["link"] = link
    
    return reviews_df

In [148]:
list_dfs = []
for link in reviews_pages:
    reviews = transform_to_dataframe(1,link)
    list_dfs.append(reviews)
    reviews = transform_to_dataframe(10, link)
    list_dfs.append(reviews)


<ipython-input-144-3b768f6b5bb8>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


InvalidArgumentException: Message: invalid argument
  (Session info: chrome=105.0.5195.125)
Stacktrace:
Backtrace:
	Ordinal0 [0x00AFDF13+2219795]
	Ordinal0 [0x00A92841+1779777]
	Ordinal0 [0x009A4100+803072]
	Ordinal0 [0x0099760A+751114]
	Ordinal0 [0x009961A8+745896]
	Ordinal0 [0x009963FD+746493]
	Ordinal0 [0x009A5A8E+809614]
	Ordinal0 [0x009FF87D+1177725]
	Ordinal0 [0x009EE7FC+1107964]
	Ordinal0 [0x009FF192+1175954]
	Ordinal0 [0x009EE616+1107478]
	Ordinal0 [0x009C7F89+950153]
	Ordinal0 [0x009C8F56+954198]
	GetHandleVerifier [0x00DF2CB2+3040210]
	GetHandleVerifier [0x00DE2BB4+2974420]
	GetHandleVerifier [0x00B96A0A+565546]
	GetHandleVerifier [0x00B95680+560544]
	Ordinal0 [0x00A99A5C+1808988]
	Ordinal0 [0x00A9E3A8+1827752]
	Ordinal0 [0x00A9E495+1827989]
	Ordinal0 [0x00AA80A4+1867940]
	BaseThreadInitThunk [0x75ABFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77937A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77937A6E+238]


In [151]:
len(list_dfs)

176

In [152]:
reviews_df = pd.concat(list_dfs)

In [31]:
#reviews_df.to_csv("reviews_to_be_cleaned.csv",index=False)
reviews_df = pd.read_csv("reviews_to_be_cleaned.csv")

In [39]:
reviews_df.loc[reviews_df["rating"] == 1]

,review,rating,link
0,It bugs me that this movie is rated so high- n...,1,https://www.imdb.com/title/tt0111161/reviews?r...
1,Shawshank is on TNT approximately five million...,1,https://www.imdb.com/title/tt0111161/reviews?r...
2,awful. a dreadful disgrace. the film is the mo...,1,https://www.imdb.com/title/tt0111161/reviews?r...
3,This film is nothing but one cliche after anot...,1,https://www.imdb.com/title/tt0111161/reviews?r...
4,The basic story for this dreadful movie is And...,1,https://www.imdb.com/title/tt0111161/reviews?r...
...,...,...,...
42025,A total waste of time! The main reason why som...,1,https://www.imdb.com/title/tt10886166/reviews?...
42026,Worst dialogue and worst everything. Anti-roma...,1,https://www.imdb.com/title/tt10886166/reviews?...
42027,What were Netflix even thinking? This is the w...,1,https://www.imdb.com/title/tt10886166/reviews?...
42028,This has to be thee worst film I've ever seen ...,1,https://www.imdb.com/title/tt10886166/reviews?...


In [42]:
number_of_element_to_delete = reviews_df.loc[reviews_df["rating"] == 10].shape[0] - reviews_df.loc[reviews_df["rating"] == 1].shape[0]
reviews_df.drop(reviews_df.loc[reviews_df["rating"] == 10].sample(n=number_of_element_to_delete).index, inplace=True)

In [45]:
reviews_df.rating.value_counts()

10    13313
1     13313
Name: rating, dtype: int64

In [48]:
reviews_df.loc[reviews_df["rating"] == 1, ["rating"]] = 0
reviews_df.loc[reviews_df["rating"] == 10, ["rating"]] = 1

In [53]:
reviews_df.rename(columns={"rating":"sent"}, inplace=True)

In [55]:
reviews_df.to_csv("data.csv", sep="|", index=False)